# KoBERT finetuning

In [1]:
# 파이썬 버전을 3.7.0으로 변경
!wget https://www.python.org/ftp/python/3.7.0/Python-3.7.0.tgz
!tar xvfz Python-3.7.0.tgz
!Python-3.7.0/configure
!make
!sudo make install

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
Compiling '/usr/local/lib/python3.7/idlelib/idle_test/test_calltips.py'...
Compiling '/usr/local/lib/python3.7/idlelib/idle_test/test_codecontext.py'...
Compiling '/usr/local/lib/python3.7/idlelib/idle_test/test_colorizer.py'...
Compiling '/usr/local/lib/python3.7/idlelib/idle_test/test_config.py'...
Compiling '/usr/local/lib/python3.7/idlelib/idle_test/test_config_key.py'...
Compiling '/usr/local/lib/python3.7/idlelib/idle_test/test_configdialog.py'...
Compiling '/usr/local/lib/python3.7/idlelib/idle_test/test_debugger.py'...
Compiling '/usr/local/lib/python3.7/idlelib/idle_test/test_delegator.py'...
Compiling '/usr/local/lib/python3.7/idlelib/idle_test/test_editmenu.py'...
Compiling '/usr/local/lib/python3.7/idlelib/idle_test/test_editor.py'...
Compiling '/usr/local/lib/python3.7/idlelib/idle_test/test_grep.py'...
Compiling '/usr/local/lib/python3.7/idlelib/idle_test/test_help.py'...
Compiling '/usr/local/lib/python3.7/idlelib/idle_test/test_help_a

In [2]:
!python --version

Python 3.7.0


In [3]:
# 필요 패키지 설치
!pip install mxnet
!pip install gluonnlp
!pip install pandas
!pip install tqdm
!pip install sentencepiece
!pip install transformers
!pip install torch
!pip install openpyxl

# /usr/local/lib/python3.10/dist-packages/mxnet/numpy/utils.py:37: FutureWarning: In the future `np.bool` will be defined as the corresponding NumPy scalar. bool = onp.bool 해결
!pip install numpy==1.23.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 33.6 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.3
    Uninstalling graphviz-0.20.3:
      Successfully uninstalled graphviz-0.20.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 344.5/344.5 kB 10.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp310-cp310-linux_x86_64.whl size=661751 sha256=e182be222382a4a3311ce5e798cce1d1cd91a7f04bfce279475b0af35a3ead86
  Stored in directory: /root/.cache/pip/wheels/1a/1e/0d/99f55911d90f2b95b9f7c176d5813ef3622894a4b30fde6bd3
Successfully built gluonnlp
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x8

In [1]:
# KoBERT 깃허브에서 필요한 파일 불러오기
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-92ikl25k/kobert-tokenizer_c9076efcc01e4ed2a5ed439b2476051e
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-92ikl25k/kobert-tokenizer_c9076efcc01e4ed2a5ed439b2476051e
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
  Created wheel for kobert_tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4633 sha256=6f3506fef2cabcdfeda7a7da5394b8b1f62d9ee30dee1de4fd552d5fc0ef905d
  Stored in directory: /tmp/pip-ephem-wheel-cache-302jkipe/wheels/e9/1a/3f/a864970e8a169c176befa3c4a1e07aa612f69195907a4045fe
Successfully built kobert_tokenizer


In [2]:
import torch
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import gluonnlp as nlp
import pandas as pd

In [3]:
from torch import nn
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm_notebook
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
from google.colab import drive
from sklearn.model_selection import train_test_split

In [4]:
# # CPU
# device = torch.device("cpu")

# GPU
device = torch.device("cuda:0")

In [25]:
# 구글 drive와 연동하여 학습에 이용할 파일 불러옴
drive.mount('/content/drive/')
data_input = pd.read_csv('/content/drive/MyDrive/junho/data/pretreated_data.csv')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [26]:
# 데이터를 제대로 불러왔는지 확인
data_input.sample(n=5)

,Text,Emotion
241187,배송이 빠르고 가격도 저렴하고 착욕하기 편해서 좋아요.상품도좋아요.재구매할 의향이 ...,긍정
97696,제품이 좋지 않은 것 같습니다. 다른 곳보다 비싸게 주고 산 것 같습니다. 사은품도...,긍정
206183,화질이 뿌옇지 않고 색도 뚜렷해서 마음에 들어요 각도도 넓어서 시야가 트이고 만족합...,긍정
136976,써보면 알죠! 인플레이션 시대에는 망설이는것도 돈 낭비되는거에요. 청소기가 자동으로...,긍정
243026,저렴할때 구입했습니다 주위에 다들좋다고해서 지금 써본지 얼마 안되었지만 상쾌한지는 ...,긍정


In [27]:
def getEmotion(emotion):
  if emotion=="부정":
    return "0"
  elif emotion=="중립":
    return "1"
  elif emotion=="긍정":
    return "2"

In [28]:
# 읽어온 학습 데이터를 리스트로 변환
data_list = []
for t, c in zip(data_input['Text'], data_input['Emotion']):
  data_list.append([t, getEmotion(c)])

In [29]:
print(len(data_list))
print(data_list[0])
print(data_list[-1])

267998
[' 7로 나온다든데 아니었나', '2']
['알쏭달쏭하다', '0']


In [30]:
dataset_train, dataset_validation = train_test_split(data_list, test_size=0.01, random_state=1)

In [31]:
print(len(dataset_train))
print(len(dataset_validation))

print(dataset_train[-1])
print(dataset_validation[-1])

265318
2680
['브러쉬 부분이 됐다 안됐다 하네요 제품이 뭐 이렇죠 정가에 제 가격 주고 샀는데 너무 짜증나네요 이게 말이되나 ㅡㅡ 다신 OO쇼핑 안삼', '2']
['소재가 부드럽고 가벼워서 좋네요 사이즈도 딱맞고 서방이 편하다니 굿', '2']


In [45]:
# 파라미터 정보
max_len = 64
batch_size = 128
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [33]:
# 학습/테스트 데이터 전처리를 위한 클래스
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, vocab=vocab, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))


In [34]:
# 데이터셋 토큰화
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')
tok = tokenizer.tokenize

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [35]:
# 각 데이터가 BERT 모델의 입력으로 들어갈 수 있도록 tokenization, int encoding, padding하는 함수
data_train = BERTDataset(dataset_train, 0, 1, tok, vocab, max_len, True, False)
data_validation = BERTDataset(dataset_validation, 0, 1, tok, vocab, max_len, True, False)

In [36]:
# torch 형식의 dataset을 만들어 입력 데이터셋의 전처리 마무리
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size = batch_size, num_workers = 5)
validation_dataloader = torch.utils.data.DataLoader(data_validation, batch_size = batch_size, num_workers = 5)

In [37]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes = 3,   # 감정 클래스 수로 조정
                 dr_rate = None,
                 params = None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p = dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device),return_dict = False)
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [38]:
# BERT  모델 불러오기
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)

In [39]:
# optimizer 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [40]:
# schedule 설정
loss_fn = nn.CrossEntropyLoss() # 다중 분류를 위한 loss function
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [41]:
# 체크포인트 변수
checkpoint_path = '/content/drive/MyDrive/junho/checkpoint/'
checkpoint_file = 'emotion_classification_model_checkpoint.pt'

In [42]:
import os

# 체크포인트 저장 함수
def save_checkpoint(model, optimizer, epoch, path=checkpoint_path+checkpoint_file):
  torch.save({
      'epoch': epoch,
      'model_state_dict': model.state_dict(),
      'optimizer_state_dict': optimizer.state_dict(),
  }, path)
  print(f"체크포인트 저장 - epoch {epoch}")

# 체크포인트 로드 함수
def load_checkpoint(model, optimizer, path=checkpoint_path+checkpoint_file):
  if not os.path.isfile(path):
    return 0
  checkpoint = torch.load(path)
  model.load_state_dict(checkpoint['model_state_dict'])
  optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
  epoch = checkpoint['epoch']
  print(f"체크포인트 로드 -  epoch {epoch}")
  return epoch

In [43]:
# 정확도 측정 함수
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [46]:
# 학습 개시
start = load_checkpoint(model, optimizer)
for e in range(start, num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        # if batch_id % log_interval == 0:
        #     print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))

    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(validation_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
    save_checkpoint(model, optimizer, e+1)

체크포인트 로드 -  epoch 5


In [47]:
# 모델을 저장할 폴더 생성
path = '/content/drive/MyDrive/junho/models/'
if not os.path.isdir(path):
  os.mkdir(path)

os.chdir(path)
os.getcwd()

'/content/drive/MyDrive/junho/models'

In [48]:
# 모델 이름
pt_name = 'emotion_classification_model.pt'
pkl_name = 'emotion_classification_model.pkl'

In [49]:
# 생성한 모델을 폴더에 저장
import joblib

torch.save(model, path + pt_name)
joblib.dump(model, path + pkl_name)

['/content/drive/MyDrive/junho/models/emotion_classification_model.pkl']

In [50]:
# 저장한 모델을 불러오기
pt_model = torch.load(path + pt_name)
pkl_model = joblib.load(path + pkl_name)

In [51]:
# 실수를 치역으로 한 가중치 값을 softmax함수를 사용하여 텍스트를 확률값으로 변환
def new_softmax(a) :
    c = np.max(a)         # 최댓값
    exp_a = np.exp(a-c)   # 각각의 원소에 최댓값을 뺀 값에 exp를 취한다. (이를 통해 overflow 방지)
    sum_exp_a = np.sum(exp_a)
    y = (exp_a / sum_exp_a) * 100
    return np.round(y, 3)

In [52]:
# 번호에 따른 태그 출력 메서드
tags = ["부정", "중립", "긍정"]

In [53]:
def predict(predict_sentence):
    data = [predict_sentence, '0']
    dataset_another = [data]
    another_test = BERTDataset(dataset_another, 0, 1, tok, vocab, max_len, True, False) # 토큰화한 문장
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)

    pt_model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = pt_model(token_ids, valid_length, segment_ids)
        test_eval=[]

        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()
            min_v = min(logits)
            total = 0
            probability = []
            logits = np.round(new_softmax(logits), 3).tolist()
            for l, logit in enumerate(logits):
                probability.append(tags[l] + '(' + str(np.round(logit, 3)) + ')')
            probability.append(tags[np.argmax(logits)])
            print(probability)
    return probability

In [54]:
# 질문에 0 입력 시 종료
end = 1
while end == 1 :
    sentence = input("하고싶은 말을 입력해주세요 : ")
    if sentence == "0" :
        break
    print(predict(sentence))
    print("\n")

하고싶은 말을 입력해주세요 : 식재까지 뻗어있는, 개인적으로 최근 가장 흥미롭게 보고 있는 블로그. 
['부정(0.777)', '중립(10.0)', '긍정(89.223)', '긍정']
['부정(0.777)', '중립(10.0)', '긍정(89.223)', '긍정']


하고싶은 말을 입력해주세요 : 식품MD로 일할 당시 부문장님께서 '전문가면 식재부터 알아야지'라고 말씀하셨었는데, 이 블로그가 딱 그 적확한 예시가 될 수 있겠다.
['부정(1.815)', '중립(95.666)', '긍정(2.519)', '중립']
['부정(1.815)', '중립(95.666)', '긍정(2.519)', '중립']


하고싶은 말을 입력해주세요 : 이번 네이버 인플루언서 검색에소 선정된 '깐깐한 엘랸씨'
['부정(96.383)', '중립(3.391)', '긍정(0.226)', '부정']
['부정(96.383)', '중립(3.391)', '긍정(0.226)', '부정']


하고싶은 말을 입력해주세요 : 이번 네이버 인플루언서 검색에소 선정된 '깐엘랸씨'
['부정(1.578)', '중립(93.72)', '긍정(4.702)', '중립']
['부정(1.578)', '중립(93.72)', '긍정(4.702)', '중립']




KeyboardInterrupt: Interrupted by user